In [1]:
%pip install numpy pandas backtesting joblib

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

import joblib

import numpy as np
from backtesting import Backtest, Strategy
from backtesting.test import BTCUSD




/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

### Define Backtest Dataser

In [3]:
# Load the dataset (which contains more features)
backtest_dataset = pd.read_csv("../datasets/BTCUSDT_backtest_features.csv") 




In [ ]:
hmmodel = joblib.load("../models/hmm.pkl")

training_df = pd.read_csv("../datasets/BTCUSDT_train_features.csv")

volume_mean = training_df['volume'].mean()
volume_std = training_df['volume'].std()

volume_mean = training_df['volume'].mean()
volume_std = training_df['volume'].std()

rsi_mean = training_df['rsi'].mean()
rsi_std = training_df['rsi'].std()

macd_mean = training_df['macd'].mean()
macd_std = training_df['macd'].std()

ema12_mean = training_df['ema_12'].mean()
ema12_std = training_df['ema_12'].std()

ema26_mean = training_df['ema_26'].mean()
ema26_std = training_df['ema_26'].std()

sma20_mean = training_df['sma_20'].mean()
sma20_std = training_df['sma_20'].std()

volatility_mean = training_df['volatility'].mean()
volatility_std = training_df['volatility'].std()

def HMM_predictor(yesterday_close, today_index):

    today_df = backtest_dataset.iloc[today_index]

    today_vol = today_df['volume']
    today_rsi = today_df['rsi']
    today_macd = today_df['macd']
    today_ema12 = today_df['ema_12']
    today_ema26 = today_df['ema_26']
    today_sma20 = today_df['sma_20']
    today_volatility = today_df['volatility']
    today_close = today_df['close']
    
    # Calculate features
    log_return = np.log(today_close/ yesterday_close)
    volume_norm = (today_vol - volume_mean) / volume_std
    rsi_norm = (today_rsi - rsi_mean) / rsi_std
    macd_norm = (today_macd - macd_mean) / macd_std
    ema12_norm = (today_ema12 - ema12_mean) / ema12_std
    ema26_norm = (today_ema26 - ema26_mean) / ema26_std
    sma20_norm = (today_sma20 - sma20_mean) / sma20_std
    volatility_norm = (today_volatility - volatility_mean) / volatility_std

    # Format as 2D array
    new_obs = np.array([[log_return, volume_norm, rsi_norm, macd_norm, ema12_norm, ema26_norm, sma20_norm, volatility_norm]])
    # new_obs = np.array([[log_return, volume_norm]])

    # Predict hidden state
    state_today = hmmodel.predict(new_obs)

    return state_today[0]

In [5]:

# class HMMStrategy(Strategy):
#     def init(self):
#         self.data_close = self.data.Close
#         self.data_volume = self.data.Volume

#     def next(self):
#         print("Current position: ", self.position)
#         if len(self.data_close) < 2:
#             return  # Not enough data
        

#         # Get today's and yesterday's close and today's volume
#         today_close = self.data_close[-1]
#         yesterday_close = self.data_close[-2]
#         today_vol = self.data_volume[-1]

#         # Predict state using HMM
#         predicted_state = HMM_predictor(today_close, yesterday_close, today_vol)
#         print(f"Predicted state: {predicted_state}")

#         # Define your action based on predicted state (assume 2 = bullish, 0 = bearish, 1 = neutral)
#         if predicted_state == 1:
#             # Bullish -> enter long
#             if not self.position:
#                 self.buy()
#                 print("Buying")
#         elif predicted_state in [0,2]:
#             # Bearish -> close long if exists
#             if self.position:
#                 self.position.close()
#                 print("Closing position")


In [6]:
class HMMStrategy(Strategy):
    def init(self):
        self.holding_period = 0

    def next(self):
        today_index = self.data.index[-1]  
        yesterday_close = self.data.Close[-2]

        predicted_state = HMM_predictor(yesterday_close, today_index)
        print(f"Predicted state: {predicted_state}")
        
        print("Self holding period: ", self.holding_period)
        if self.position:
            self.holding_period += 1
        else:
            self.holding_period = 0

      
        if self.holding_period >= 30 or predicted_state == 1 or predicted_state == 2 : ## Bearish or neutral or holding period is over
            if self.position:
                self.position.close()
                self.holding_period = 0
                print("Closing position")
        elif predicted_state == 0: ## Bullish
            if not self.position:
                self.buy()
                print("Buying")


In [7]:
# Extract the relevant columns for backtesting
backtest_df = pd.DataFrame(backtest_dataset.iloc[:, :6])

# Rename the columns to match Backtesting.py requirements
backtest_df.rename(columns={
    'timestamp': 'Date',
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'volume': 'Volume'
}, inplace=True)

# Convert the 'Date' column to datetime
backtest_df['Date'] = pd.to_datetime(backtest_df['Date'])


bt = Backtest(backtest_df, HMMStrategy, cash=1000000, commission=0.0006)
stats = bt.run()
# bt.plot()

# print(BTCUSD)

# bt = Backtest(BTCUSD, HMMStrategy, cash=100000, commission=0.0006)
# stats = bt.run()
# bt.plot()


/var/folders/zp/ypnn7zh17bq2rm1j9d6glplw0000gn/T/ipykernel_35905/1118971592.py:18: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(backtest_df, HMMStrategy, cash=1000000, commission=0.0006)


Predicted state: 1
Self holding period:  0
Predicted state: 1
Self holding period:  0
Predicted state: 1
Self holding period:  0
Predicted state: 1
Self holding period:  0
Predicted state: 1
Self holding period:  0
Predicted state: 1
Self holding period:  0
Predicted state: 1
Self holding period:  0
Predicted state: 1
Self holding period:  0
Predicted state: 1
Self holding period:  0
Predicted state: 1
Self holding period:  0
Predicted state: 1
Self holding period:  0
Predicted state: 1
Self holding period:  0
Predicted state: 1
Self holding period:  0
Predicted state: 1
Self holding period:  0
Predicted state: 1
Self holding period:  0
Predicted state: 1
Self holding period:  0
Predicted state: 1
Self holding period:  0
Predicted state: 1
Self holding period:  0
Predicted state: 1
Self holding period:  0
Predicted state: 1
Self holding period:  0
Predicted state: 1
Self holding period:  0
Predicted state: 1
Self holding period:  0
Predicted state: 1
Self holding period:  0
Predicted s

In [8]:
print (stats)

Start                                     0.0
End                                   26970.0
Duration                              26970.0
Exposure Time [%]                         0.0
Equity Final [$]                    1000000.0
Equity Peak [$]                     1000000.0
Return [%]                                0.0
Buy & Hold Return [%]                48.75653
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Alpha [%]                                 0.0
Beta                                      0.0
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                  0.0
Win Rate [%]                              NaN
Best Trade [%]                    